In [5]:
import sys

if True:
    # !pip install -U "jax[cuda12]"
    !pip install jaxkan --no-deps
    !pip install tqdm

if True:
    """
    import os
    os.environ['XLA_FLAGS'] = (
        '--xla_gpu_enable_triton_softmax_fusion=true '
        '--xla_gpu_triton_gemm_any=True '
        '--xla_gpu_enable_async_collectives=true '
        '--xla_gpu_enable_latency_hiding_scheduler=true '
        '--xla_gpu_enable_highest_priority_async_stream=true '
    )
    """
#    sys.path.append('/kaggle/input/utilspikan')

from flax import linen as nn
from flax.training import train_state
from flax.linen.initializers import glorot_normal
from typing import Callable

import jax
import jax.numpy as jnp
import numpy as np
import optax

from jaxkan.models.KAN import KAN

from tqdm import tqdm
import matplotlib.pyplot as plt

jax.devices()

from Utils import *

r = KeyHandler(0)

/Users/dcr/miniconda3/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [23]:
# Defining the MLP_fourier model initialization
glorot = nn.initializers.glorot_normal()
class FourierFeats(nn.Module):
    num_output: int
    
    std = 10
    mean = 0
    
    @nn.compact
    def __call__(self, x):
        B = self.param(
            'B', lambda rng, shape: self.mean + jax.random.normal(rng, shape) * self.std,
           (x.shape[-1],self.num_output//2)
        )
        bias = self.param(
            'bias', lambda rng, shape: jax.random.normal(rng, shape)*0,
            (self.num_output//2,)
        )
        
        x = jnp.matmul(x,B)
        x = jnp.concatenate([jnp.cos(2*jnp.pi*(x+bias)), jnp.sin(2*jnp.pi*(x+bias))], axis=-1)
            
        return x
    
layer = FourierFeats(num_output=64)
collocs = jnp.ones((64,2))
variables = layer.init(r.key(), collocs)
variables
layer.apply(variables, collocs).shape

(64, 64)